In [1]:
import cleanText
from tensorflowEval import ClassificationReport
from DisasterDetector import DisasterDetector

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

In [3]:
import tensorflow_hub as hub

In [7]:
df_train = pd.read_csv("../data/raw/train.csv",dtype={'id': np.int16, 'target': np.int8})
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
#df_train['text_cleaned'] = df_train['text'].apply(lambda s : cleanText.clean(s))
df_train = pd.read_csv("../data/raw/trainProc.csv",dtype={'id': np.int16, 'target': np.int8})
#df_train[["id","text_cleaned","target"]].to_csv("../data/raw/trainProc.csv",index=False)
df_train.head()

,id,text_cleaned,target
0,1,Our Deeds are the Reason of this # earthquake...,1
1,4,Forest fire near La Ronge Sask . Canada,1
2,5,All residents asked to ' shelter in place ' ...,1
3,6,"13,000 people receive # wildfires evacuation ...",1
4,7,Just got sent this photo from Ruby # Alaska a...,1


In [5]:
K = 2
seed = 100
skf = StratifiedKFold(n_splits=K, random_state=seed, shuffle=True)

In [6]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 12.2 s, sys: 2.1 s, total: 14.3 s
Wall time: 14.5 s


In [ ]:
clf = DisasterDetector(bert_layer,skf, max_seq_length=27, lr=0.0001, epochs=10, batch_size=32)

clf.train(df_train)


Fold 0

Epoch 1/10
 29/119 [======>.......................] - ETA: 22:56 - loss: 0.7162 - accuracy: 0.5539